In [1]:
import pandas as pd

In [3]:
census_data = pd.read_csv("raw/CIS2017.tab", sep="\t", header=0)
census_data.columns

Index(['YEAR', 'PUMFID', 'PERSONID', 'FWEIGHT', 'PROV', 'USZGAP', 'MBMREGP',
       'AGEGP', 'SEX', 'MARST',
       ...
       'DWLTYP', 'DWTENR', 'REPA', 'SUIT', 'MORTG', 'MORTGM', 'CONDMP',
       'RENTM', 'CHNEED', 'VERDATE'],
      dtype='object', length=194)